In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import cv2
import os
from keras.utils import np_utils

In [2]:
IMG_SIZE = 100
x = [] #features
y = [] #labels
source = 'left_eyes_data'
for filename in os.listdir(source):
    string = filename.split(".")
    img = cv2.imread(os.path.join(source,filename))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    x.append(img)
    y.append([int(string[0]), int(string[1])])

In [17]:
x = np.array(x).reshape(len(x), 100, 100, 1)
x = x.astype('float32')
x /= 255

In [18]:
y = np.array(y)

In [19]:
x[0]

array([[[4.7040735e-06],
        [4.6437649e-06],
        [4.5834563e-06],
        ...,
        [4.7040735e-06],
        [4.8246907e-06],
        [4.8849993e-06]],

       [[4.6437649e-06],
        [4.5834563e-06],
        [4.5231477e-06],
        ...,
        [4.6437649e-06],
        [4.7643821e-06],
        [4.8246907e-06]],

       [[4.5834563e-06],
        [4.5231477e-06],
        [4.4025301e-06],
        ...,
        [4.5231477e-06],
        [4.5834563e-06],
        [4.6437649e-06]],

       ...,

       [[6.6942580e-06],
        [6.6942580e-06],
        [6.7545666e-06],
        ...,
        [6.3324060e-06],
        [6.2720974e-06],
        [6.2720974e-06]],

       [[6.7545666e-06],
        [6.7545666e-06],
        [6.8148752e-06],
        ...,
        [6.3927146e-06],
        [6.3324060e-06],
        [6.3324060e-06]],

       [[6.8148752e-06],
        [6.8148752e-06],
        [6.8751838e-06],
        ...,
        [6.3927146e-06],
        [6.3324060e-06],
        [6.3324060e-06]]

In [20]:
y[0]

array([0, 0])

In [21]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [22]:
print("x-train: ", x_train.shape)
print("x-test: ", x_test.shape)
print("y-train: ", y_train.shape)
print("y-test: ", y_test.shape)

x-train:  (19588, 100, 100, 1)
x-test:  (8396, 100, 100, 1)
y-train:  (19588, 2)
y-test:  (8396, 2)


In [10]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten

In [23]:
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(100, 100, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(32, input_dim=10000, kernel_initializer='he_uniform', activation='relu'))
model.add(Dense(2, kernel_initializer='he_uniform'))

In [24]:
model.compile(loss='mae', optimizer='adam', metrics=['accuracy'])

In [25]:
model.fit(x_train, y_train, batch_size=32, validation_data=(x_test, y_test), steps_per_epoch=len(x_train)//32, epochs=3)

Epoch 1/3
612/612 [==============================] - 1252s 2s/step - loss: 294.4173 - accuracy: 0.7697 - val_loss: 283.5503 - val_accuracy: 0.7670
Epoch 2/3
612/612 [==============================] - 943s 2s/step - loss: 285.1762 - accuracy: 0.7708 - val_loss: 283.7749 - val_accuracy: 0.7670
Epoch 3/3
612/612 [==============================] - 996s 2s/step - loss: 285.7248 - accuracy: 0.7706 - val_loss: 290.9488 - val_accuracy: 0.7670


In [26]:
y_test_pred = model.predict(x_test)

In [27]:
y_test_pred

array([[569.2554 , 202.46005],
       [569.1974 , 202.43936],
       [569.18506, 202.43512],
       ...,
       [569.2493 , 202.4579 ],
       [569.23254, 202.45192],
       [569.187  , 202.43571]], dtype=float32)

In [28]:
y_test

array([[ 778,    0],
       [ 650,   76],
       [ 842,  589],
       ...,
       [1021,  200],
       [ 100,  600],
       [ 541,    1]])

In [30]:
model_json = model.to_json()
with open("cnn.json", "w") as json_file:
    json_file.write(model_json)

model.save_weights("cnn.h5")
print("Saved model to disk")

Saved model to disk


In [1]:
from keras.models import model_from_json
from tkinter import * 
import threading
import pandas as pd
import numpy as np
import cv2
import os

json_file = open('cnn.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

loaded_model.load_weights("cnn.h5")
print("Loaded model from disk")

Loaded model from disk


In [ ]:
x_predict = 0
y_predict = 0

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades +'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades +'haarcascade_eye.xml')

cap = cv2.VideoCapture(0)
_, img = cap.read()

def show_frame():
    global img, x_predict, y_predict
    while True:
        _, img = cap.read()
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(img, 1.3, 5)
        for (x,y,w,h) in faces:
            roi_gray = img[y:y+h, x:x+w]
            eyes = eye_cascade.detectMultiScale(roi_gray, 1.3, 5) # detect eyes
            width = np.size(roi_gray, 1) # get face frame width
            height = np.size(roi_gray, 0) # get face frame height
            for (x, y, w, h) in eyes:
                if y > height / 2:
                    pass
                eyecenter = x + w / 2  # get the eye center
                if eyecenter < width * 0.5:
                    left_eye = roi_gray[y:y + h, x:x + w]
                    left_eye = cv2.resize(left_eye, dsize=(100, 100))
                    cv2.imshow("eye", left_eye)
                    left_eye = np.array(left_eye).reshape(-1, 100, 100, 1)
                    left_eye = left_eye.astype('float32')
                    left_eye /= 255
                    prediction = loaded_model.predict([left_eye])
                    x_predict = int(prediction[0][0])
                    y_predict = int(prediction[0][1])
        interrupt = cv2.waitKey(10)
        if interrupt & 0xFF == 27: # esc key
            break

def tkinter_frame():
    global myCanvas
    frame = Tk()  
    frame.attributes('-fullscreen',True) 
    
    myCanvas = Canvas(frame)
    myCanvas.pack(fill=BOTH, expand=True)
    
    frame.mainloop()

def pointer():
    global x_predict, y_predict
    while True:
        create_circle(x_predict, y_predict, 20)
        
def create_circle(x, y, r): #center coordinates, radius
    global myCanvas, dot
    x0 = x - r
    y0 = y - r
    x1 = x + r
    y1 = y + r
    try:
        if dot:
            myCanvas.delete(dot)
    except:
        pass
    try:
        dot = myCanvas.create_oval(x0, y0, x1, y1, fill="red")  
    except:
        pass
        

t1 = threading.Thread(target=show_frame)
t2 = threading.Thread(target=tkinter_frame)
t3 = threading.Thread(target=pointer)

# starting thread 1
t1.start()
# starting thread 2
t2.start()
t3.start()

# wait until thread 1 is completely executed
# t1.join()
# wait until thread 2 is completely executed
t2.join() 
t1.join()